In [1]:
import cv2
import numpy as np

# Generate a numpy array from a video
1. Load the video and read the metadata
1. Dowsample the video at the targeted fps and resize the frames
1. Save the numpy array :3D + 3D numpy with colors or 3D if gray


### Load the video and read the metadata

In [2]:
path_videos = "../data/videos/"
video_name = "Tom Hanks' Amazing Clint Eastwood Impression - The Graham Norton Show.mp4"
video_in = path_videos + video_name

In [9]:
# Read video properties

SHOW_DWN_VIDEO = True  # To visualise the downsized video
USE_COLOR = True  # To save in cv2 RGB format

video = cv2.VideoCapture(video_in)

frame_count = video.get(cv2.CAP_PROP_FRAME_COUNT)
frame_width = video.get(cv2.CAP_PROP_FRAME_WIDTH)
frame_height = video.get(cv2.CAP_PROP_FRAME_HEIGHT)
fps = video.get(cv2.CAP_PROP_FPS)

print("Frame per Second : {:.2f}".format(fps))
print("Frame size : (width,height) = ({},{})".format(frame_width, frame_height))
print("Frame count : {}".format(frame_count))

Frame per Second : 29.97
Frame size : (width,height) = (1280.0,720.0)
Frame count : 4562.0


## Dowsample the video at the targeted fps and resize the frames

In [10]:

# define the fps for the numpy array
fps_np = 1.0

# rounded to get an uniform sampling or float to get a better approximation of the fps_np
# but loose the dimension of the array
dwn_fps = round(fps/fps_np)

# shrink factor to shrink the frames
# the dimension of the resized_frame (width_np, height_np)
# can be set manually

shrink_factor = 4.
width_np = int(frame_width/shrink_factor)
height_np = int(frame_height/shrink_factor)
dim_np = (width_np, height_np)

# number of frames after donsampling
# the +1 because at the init we start at frame 0
frame_count_np = int(frame_count/dwn_fps)+1

# np array size
if USE_COLOR:
    video_np = np.zeros((frame_count_np, height_np, width_np, 3))
else:
    video_np = np.zeros((frame_count_np, height_np, width_np))


# init the first frame is the frame zero
success = True
frame_number_flt = 0
frame_number = round(frame_number_flt)
array_idx = 0

while success and frame_number < frame_count:

    # take the undersampled frame
    video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    success, frame = video.read()

    # resize frame to the dimension of the array dim_np
    resized_frame = cv2.resize(
        frame, (width_np, height_np), interpolation=cv2.INTER_AREA)

    if USE_COLOR == False:
        resized_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)

    if SHOW_DWN_VIDEO:
        cv2.imshow('resized frame', resized_frame)
        aa = cv2.waitKey(1)

    # save in an array
    video_np[array_idx, :] = resized_frame

    # increment the frames and numpy array index
    frame_number_flt += dwn_fps
    frame_number = round(frame_number_flt)
    array_idx += 1

In [11]:
# clean up 
video.release()

In [13]:
# close the window
if SHOW_DWN_VIDEO:
    cv2.waitKey(1000)
    cv2.destroyAllWindows()

## Save the numpy array :3D + 3D numpy with colors or 3D if gray

In [14]:
# save the numpy array 
path_arrays = "../data/arrays/"
array_name = 'Tom_Hanks_GNS_' + str(int(fps_np)) + 'fps'
if USE_COLOR:
    array_name += '_RGB'
else:
    array_name += '_Gray'
file_out =  path_arrays + array_name
np.save(file = file_out, arr = video_np)